In [ ]:
from arc.main import ARC
from arc.common import save_yaml_file
from arc.species import ARCSpecies

from arkane.input import load_input_file

In [ ]:
path = '/home/alon/Dropbox (MIT)/scripts/DRGScripts/data/Decalin_network0_reduced.py'

In [ ]:
job_list, reaction_dict, species_dict, transition_state_dict, network_dict, level_of_theory =\
    load_input_file(path)


In [ ]:
arc_species = list()

for spc in species_dict.values():
    # modify this condition to choose which species to pass down to ARC
    # if 'group' in spc.thermo.comment or 'radical' in spc.thermo.comment:
    if 'DFT_QCI_thermo' not in spc.thermo.comment and 'primaryThermoLibrary' not in spc.thermo.comment:
        mul = spc.molecule[0].multiplicity
        arc_species.append(ARCSpecies(label=spc.label,
                                      mol=spc.molecule[0],
                                      multiplicity=mul if mul != 3 else 1,  # in this specific case we don't allow triplets
                                      number_of_radicals=2 if mul == 3 else None,  # marke singlet bitads properly
                                     ))

In [ ]:
# Add additional species manually if needed:
arc_species.append(ARCSpecies(label='C1CC[CH]CCCC[CH]C1',
                   smiles='C1CC[CH]CCCC[CH]C1',
                   multiplicity=1,
                   number_of_radicals=2,
                  ))

print(f'Got {len(arc_species)} species')

In [ ]:
# Write an ARC inpiut file

arc_input_file_path = '/home/alon/Dropbox (MIT)/scripts/DRGScripts/data/input.yml'

level_of_theory = 'CBS-QB3'

arc_object = ARC(project='Project_1',
                 species=arc_species,
                 level_of_theory=level_of_theory,
                )

arc_object.write_input_file(path=arc_input_file_path)


In [ ]:
# This will write a detailed ARC input file with all default argumets

arc_object = ARC(project='Project_1',
                 species=arc_species,
                 level_of_theory=level_of_theory,
                )

arc_object.write_input_file(path=arc_input_file_path)

In [ ]:
# This will write a minimal (human-readable) ARC input file without unnecessary default argumets

content = {'project': 'Project_1',
           'level_of_theory': level_of_theory,
           # 'species': [spc.as_dict() for spc in arc_species],
           'species': [{'label': spc.label,
                        'smiles': spc.mol.to_smiles(),
                        'multiplicity': spc.multiplicity,
                        'number_of_radicals': spc.number_of_radicals,
                       } for spc in arc_species],
          }

save_yaml_file(path=arc_input_file_path,
               content=content,
              )